### Set autoreloading
This extension will automatically update with any changes to packages in real time

In [1]:
%load_ext autoreload
%autoreload 2

### Import packages
We'll need the `pytorch_lightning` and `nugraph` packages imported in order to train

In [2]:
import os
from pathlib import Path
import nugraph as ng
import pytorch_lightning as pl

### Set model and data to use

This allows the user to switch out different model architectures and datasets

In [3]:
Data = ng.data.NuGraphDataModule
Model = ng.models.NuGraph3

### Configure data module
Declare a data module. Depending on where you're working, you should edit the data path below to point to a valid data location.

In [4]:
nudata = Data(model=Model)

### Configure network
Declare a model. You can edit the arguments below to change the network configuration.

In [5]:
nugraph = Model(
    in_features=5,
    hit_features=128,
    nexus_features=32,
    instance_features=32,
    interaction_features=32,
    semantic_classes=nudata.semantic_classes,
    event_classes=nudata.event_classes,
    num_iters=5,
    event_head=False,
    semantic_head=True,
    filter_head=True,
    vertex_head=False,
    instance_head=True,
    use_checkpointing=True,
    lr=0.001)

In [6]:
import os

# Set environment variable for NUGRAPH_LOG
os.environ["NUGRAPH_LOG"] = "/exp/sbnd/app/users/yuhw/nugraph/log"

# Confirm the variable has been set
print(f"NUGRAPH_LOG is now set to: {os.environ['NUGRAPH_LOG']}")

NUGRAPH_LOG is now set to: /exp/sbnd/app/users/yuhw/nugraph/log


### Configure logger and callbacks
Declare a tensorboard logger and define the output directory, so we can monitor network training. Also define a callback so we can monitor learning rate evolution.

In [7]:
name = "test"
logdir = Path(os.environ["NUGRAPH_LOG"])/name
logdir.mkdir(parents=True, exist_ok=True)
logger = pl.loggers.WandbLogger(save_dir=logdir, project="nugraph3", name="test",
                                log_model="all")
callbacks = [
    pl.callbacks.LearningRateMonitor(logging_interval="step"),
    pl.callbacks.ModelCheckpoint(monitor="loss/val", mode="min"),
]

### Declare trainer and run training
First we set the training device. To train with a GPU, pass an integer  otherwise, it defaults to CPU training. We then instantiate a PyTorch Lightning trainer that we'll use for training, and then run the training stage, which iterates over all batches in the train and validation datasets to optimise model parameters, writing output metrics to tensorboard.

In [8]:
accelerator, devices = ng.util.configure_device(0)
print(f"Using accelerator={accelerator}, devices={devices}")
trainer = pl.Trainer(accelerator=accelerator,
                     devices=devices,
                     max_epochs=1,
                     logger=logger,
                     callbacks=callbacks)
trainer.fit(nugraph, datamodule=nudata)
trainer.test(datamodule=nudata)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe MIG 2g.20gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Using accelerator=gpu, devices=[0]


wandb: Currently logged in as: yuhw (yuhw-brookhaven-national-laboratory) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
/exp/sbnd/app/users/yuhw/dl-clustering/venv_eaf/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name             | Type            | Params | Mode 
-------------------------------------------------------------
0 | encoder          | Encoder         | 779    | train
1 | core_net         | NuGraphCore     | 99.7 K | train
2 | semantic_decoder | SemanticDecoder | 646    | train
3 | filter_decoder   | FilterDecoder   | 130    | train
4 | instance_decoder | InstanceDecoder | 4.3 K  | train
-------------------------------------------------------------
105 K     Trainable params
11        Non-trainable params
105 K     Total params
0.422     Total estimated model params s

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
/exp/sbnd/app/users/yuhw/dl-clustering/venv_eaf/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:149: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Restoring states from the checkpoint path at /exp/sbnd/app/users/yuhw/nugraph/log/test/nugraph3/28tv46mj/checkpoints/epoch=0-step=1.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /exp/sbnd/app/users/yuhw/nugraph/log/test/nugraph3/28tv46mj/checkpoints/epoch=0-step=1.ckpt
/exp/sbnd/app/users/yuhw/dl-clustering/venv_eaf/lib/python3.10/site-packages/torch_geometric/data/storage.py:452: UserWarning:

Unable to accurately infer 'num_nodes' from the attribute set '{'y_vtx', 'y'}'. Please explici

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃         Test metric          ┃         DataLoader 0         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       filter/loss-test       │      1.4177418947219849      │
│    filter/precision-test     │             0.0              │
│      filter/recall-test      │             0.0              │
│ instance/adjusted-rand-test  │             0.0              │
│    instance/bkg-loss-test    │      0.9996777176856995      │
│      instance/loss-test      │      3.6501784324645996      │
│ instance/potential-loss-test │      2.650500774383545       │
│          loss/test           │      6.943850517272949       │
│      semantic/loss-test      │      1.8759301900863647      │
│   semantic/precision-test    │      0.125944584608078       │
│     semantic/recall-test     │      0.125944584608078       │
└──────────────────────────────┴──────────────────────────────┘

[{'loss/test': 6.943850517272949,
  'semantic/loss-test': 1.8759301900863647,
  'semantic/recall-test': 0.125944584608078,
  'semantic/precision-test': 0.125944584608078,
  'filter/loss-test': 1.4177418947219849,
  'filter/recall-test': 0.0,
  'filter/precision-test': 0.0,
  'instance/loss-test': 3.6501784324645996,
  'instance/bkg-loss-test': 0.9996777176856995,
  'instance/potential-loss-test': 2.650500774383545,
  'instance/adjusted-rand-test': 0.0}]